In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, row_number, split, sum, when
from pyspark.sql.window import Window

# Create a SparkSession
spark = SparkSession.builder.appName("BowlerStats").getOrCreate()

# Read the CSV file into a DataFrame
df = spark.read.csv("/FileStore/tables/cricket/335982_b.csv",header=True)

# Group the data by bowling team and bowler, and count the total wickets
bowler_stats = df.groupBy("team_bowling", "bowler").agg(
    sum(when(col("kind") != "-", 1).otherwise(0)).alias("total_wickets_taken")
)

# Create a window specification to rank the bowlers within each bowling team based on total wickets
windowSpec = Window.partitionBy("team_bowling").orderBy(bowler_stats["total_wickets_taken"].desc())

# Add a rank column to the DataFrame based on the window specification
bowler_stats_with_rank = bowler_stats.withColumn("rank", row_number().over(windowSpec))

# Filter the top 5 bowlers for each bowling team
top_bowlers = bowler_stats_with_rank.filter(bowler_stats_with_rank["rank"] <= 5)

# Show the scorecard for bowlers
teams = top_bowlers.select("team_bowling").distinct().collect()

for team in teams:
    team_name = team["team_bowling"]
    team_df = top_bowlers.filter(top_bowlers["team_bowling"] == team_name).drop("rank")
    print(f"Team: {team_name}")
    team_df.show(truncate=False)


Team: Kolkata Knight Riders
+---------------------+----------+-------------------+
|team_bowling         |bowler    |total_wickets_taken|
+---------------------+----------+-------------------+
|Kolkata Knight Riders|AB Agarkar|3                  |
|Kolkata Knight Riders|SC Ganguly|3                  |
|Kolkata Knight Riders|AB Dinda  |2                  |
|Kolkata Knight Riders|LR Shukla |1                  |
|Kolkata Knight Riders|I Sharma  |1                  |
+---------------------+----------+-------------------+

Team: Royal Challengers Bangalore
+---------------------------+---------+-------------------+
|team_bowling               |bowler   |total_wickets_taken|
+---------------------------+---------+-------------------+
|Royal Challengers Bangalore|AA Noffke|1                  |
|Royal Challengers Bangalore|JH Kallis|1                  |
|Royal Challengers Bangalore|Z Khan   |1                  |
|Royal Challengers Bangalore|CL White |0                  |
|Royal Challengers Ban